In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix
pd.set_option("display.max.columns", None)

In [2]:
def reduce_mem_usage(data_df, sparse=False):
    """Reduce memory usage of Pandas DF.
    From https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
    
    :param df: Pandas DF
    :type df: :class:`pandas.DataFrame`
    :return: Pandas DF
    :rtype: :class:`pandas.DataFrame`
    """
    megabyte = 1024**2
    start_mem = data_df.memory_usage().sum()/megabyte
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')

    for col in data_df.columns:
        col_type = data_df[col].dtype

        if col_type != object:
            c_min = data_df[col].min()
            c_max = data_df[col].max()

            if str(col_type).startswith('int') or str(col_type).startswith('uint'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data_df[col] = data_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data_df[col] = data_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data_df[col] = data_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data_df[col] = data_df[col].astype(np.int64)
            elif str(col_type).startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data_df[col] = data_df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data_df[col] = data_df[col].astype(np.float32)
                else:
                    data_df[col] = data_df[col].astype(np.float64)
        else:
            data_df[col] = data_df[col].astype('category')

    if sparse:
        data_df = csr_matrix(data_df)
        end_mem = (data_df.data.nbytes + data_df.indptr.nbytes + data_df.indices.nbytes)/megabyte
    else:
        end_mem = data_df.memory_usage().sum()/megabyte
        
    percent_change = 100*(start_mem-end_mem)/start_mem
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {percent_change:.1f}%')

    return data_df

In [3]:
train_id = pd.read_csv("./data/train_identity.csv")

In [4]:
# Identify categorical columns not already binarized
string_columns = []
for column in train_id.columns:
    if train_id[column].dtype=="O":
        string_columns.append(column)

In [5]:
# Create dummy variables for categorical columns
for c in string_columns:
    dummy_df = pd.get_dummies(train_id[c], prefix=c, dummy_na=True)
    train_id = pd.concat([train_id, dummy_df], axis=1).drop(c, axis=1)

In [6]:
print(train_id.shape)
train_id.head()

(144233, 2322)


TransactionID  id_01     id_02  id_03  id_04  id_05  id_06  id_07  id_08  \
0        2987004    0.0   70787.0    NaN    NaN    NaN    NaN    NaN    NaN   
1        2987008   -5.0   98945.0    NaN    NaN    0.0   -5.0    NaN    NaN   
2        2987010   -5.0  191631.0    0.0    0.0    0.0    0.0    NaN    NaN   
3        2987011   -5.0  221832.0    NaN    NaN    0.0   -6.0    NaN    NaN   
4        2987016    0.0    7460.0    0.0    0.0    1.0    0.0    NaN    NaN   

   id_09  id_10  id_11  id_13  id_14  id_17  id_18  id_19  id_20  id_21  \
0    NaN    NaN  100.0    NaN -480.0  166.0    NaN  542.0  144.0    NaN   
1    NaN    NaN  100.0   49.0 -300.0  166.0    NaN  621.0  500.0    NaN   
2    0.0    0.0  100.0   52.0    NaN  121.0    NaN  410.0  142.0    NaN   
3    NaN    NaN  100.0   52.0    NaN  225.0    NaN  176.0  507.0    NaN   
4    0.0    0.0  100.0    NaN -300.0  166.0   15.0  529.0  575.0    NaN   

   id_22  id_24  id_25  id_26  id_32  id_12_Found  id_12_NotFound  id_12_nan  \
0    NaN    NaN    NaN    NaN   32.0            0               1          0   
1    NaN    NaN    NaN    NaN   32.0            0               1          0   
2    NaN    NaN    NaN    NaN    NaN            0               1          0   
3    NaN    NaN    NaN    NaN    NaN            0               1          0   
4    NaN    NaN    NaN    NaN   24.0            0               1          0   

   id_15_Found  id_15_New  id_15_Unknown  id_15_nan  id_16_Found  \
0            0          1              0          0            0   
1            0          1              0          0            0   
2            1          0              0          0            1   
3            0          1              0          0            0   
4            1          0              0          0            1   

   id_16_NotFound  id_16_nan  id_23_IP_PROXY:ANONYMOUS  id_23_IP_PROXY:HIDDEN  \
0               1          0                         0                      0   
1               1          0                         0                      0   
2               0          0                         0                      0   
3               1          0                         0                      0   
4               0          0                         0                      0   

   id_23_IP_PROXY:TRANSPARENT  id_23_nan  id_27_Found  id_27_NotFound  \
0                           0          1            0               0   
1                           0          1            0               0   
2                           0          1            0               0   
3                           0          1            0               0   
4                           0          1            0               0   

   id_27_nan  id_28_Found  id_28_New  id_28_nan  id_29_Found  id_29_NotFound  \
0          1            0          1          0            0               1   
1          1            0          1          0            0               1   
2          1            1          0          0            1               0   
3          1            0          1          0            0               1   
4          1            1          0          0            1               0   

   id_29_nan  id_30_Android  id_30_Android 4.4.2  id_30_Android 5.0  \
0          0              0                    0                  0   
1          0              0                    0                  0   
2          0              0                    0                  0   
3          0              0                    0                  0   
4          0              0                    0                  0   

   id_30_Android 5.0.2  id_30_Android 5.1.1  id_30_Android 6.0  \
0                    0                    0                  0   
1                    0                    0                  0   
2                    0                    0                  0   
3                    0                    0                  0   
4                    0 

In [7]:
train_id_reduced = reduce_mem_usage(train_id, sparse=True)


Memory usage of dataframe is 342.50 MB
Memory usage after optimization is: 62.76 MB
Decreased by 81.7%


## Note:
Without using sparse matrices, the `reduce_mem_usage` function returned:
```
Memory usage of dataframe is 342.50 MB
Memory usage after optimization is: 323.25 MB
Decreased by 5.6%
```

With sparse matrices, the `reduce_mem_usage` function returned:

```
Memory usage of dataframe is 342.50 MB
Memory usage after optimization is: 62.76 MB
Decreased by 81.7%
```

In [ ]:
train_transaction = pd.read_csv("./data/train_transaction.csv")